<a href="https://colab.research.google.com/github/karandomguy/email-2-faq/blob/experiments/similaritycheckquora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Pimport pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import *
import numpy as np
import pandas as pd 
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score
cv = CountVectorizer()
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import tokenize
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install zipfile36

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:


# import required modules
import zipfile
import pandas as pd
 
# read the dataset using the compression zip
df = pd.read_csv('question-pairs-dataset.zip',compression='zip')
 
# display dataset
print(df.head())

   id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? How can I solve...   
4   4     9    10  Which one dissolve in water quikly sugar, salt...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  
1  What would happen if the Indian government sto...             0  
2  How can Internet speed be increased by hacking...             0  
3  Find the remainder when [math]23^{24}[/math] i...             0  
4            Which fish would survive in salt water?             0  


In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
df["question1"]=df["question1"].str.lower()
df["question2"]=df["question2"].str.lower()

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,3,4,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...,0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,3,7,8,why am i mentally very lonely? how can i solve...,find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?,0


In [ ]:

len(df)

404351

In [ ]:
def cleaning(text):
    text = str(text)
    text = text.lower()
    text = re.sub('\s\W',' ',text)
    text = re.sub('\W,\s',' ',text)
    text = re.sub(r'[^\w]', ' ', text)
    text = re.sub("\d+", "", text)
    text = re.sub('\s+',' ',text)
    text = re.sub('[!@#$_]', '', text)
    text = text.replace("co","")
    text = text.replace("https","")
    text = text.replace(",","")
    text = text.replace("[\w*"," ")
    
    return text
df['question1'] = [cleaning(text) for text in df['question1']]
df['question2'] = [cleaning(text) for text in df['question2']]

In [ ]:
newdf=df.drop(['qid1','qid2'], axis=1)


In [ ]:
newdf.head()

,id,question1,question2,is_duplicate
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...,0
2,2,how can i increase the speed of my internet nn...,how can internet speed be increased by hacking...,0
3,3,why am i mentally very lonely how can i solve it,find the remainder when math math is divided by,0
4,4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0


*Jaccard Similarity*

In [ ]:
def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two lists """
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

In [ ]:
val=0
count=0
for i in range(20000):
    sentences = [newdf['question1'][i],
    newdf['question2'][i]]
    sentences = [sent.lower().split(" ") for sent in sentences]
    if jaccard_similarity(sentences[0], sentences[1])<0.5:
      ans=True
    else:
      ans=False
    if ans==newdf['is_duplicate'][i]:
      count+=1
print("accuracy=",count/20000)

accuracy= 0.346


*Levenshtein Distance*

In [ ]:
count=0
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        s1, s2 = s2, s1

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]/float(len(s1))
for i in range(10000):
  s1 = newdf['question1'][i]
  s2 = newdf['question2'][i]
  example_1 = (s1, s2)
  if levenshtein(*example_1)<0.5:
      ans=True
  else:
      ans=False
  if ans==newdf['is_duplicate'][i]:
    count+=1
print("accuracy=",count/10000)

accuracy= 0.665


*Cosine Similarity*

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
count=0
# Program to measure the similarity between
# two sentences using cosine similarity.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# X = input("Enter first string: ").lower()
# Y = input("Enter second string: ").lower()



for i in range(50000):
  X = newdf['question1'][i]
  Y = newdf['question2'][i]

  # tokenization
  X_list = word_tokenize(X)
  Y_list = word_tokenize(Y)

  # sw contains the list of stopwords
  sw = stopwords.words('english')
  l1 =[];l2 =[]

  # remove stop words from the string
  X_set = {w for w in X_list if not w in sw}
  Y_set = {w for w in Y_list if not w in sw}

  # form a set containing keywords of both strings
  rvector = X_set.union(Y_set)
  for w in rvector:
    if w in X_set: l1.append(1) # create a vector
    else: l1.append(0)
    if w in Y_set: l2.append(1)
    else: l2.append(0)
  c = 0

  # cosine formula
  for i in range(len(rvector)):
      c+= l1[i]*l2[i]
  if float((sum(l1)*sum(l2))**0.5)!=0:
    cosine = c / float((sum(l1)*sum(l2))**0.5)
  else:
    continue;
  if cosine<0.5:
      ans=True
  else:
      ans=False
  if ans==newdf['is_duplicate'][i]:
    count+=1
print(count)
print("accuracy=",count/50000)

29159
accuracy= 0.58318


LSTM


In [ ]:
def load_data(df):
    question1 = df["question1"].astype(str).values
    question2 = df["question2"].astype(str).values
    # combined: to get the tokens
    df['combined'] = df["question1"] + df["question2"]
    labels = df["is_duplicate"].values
    return question1, question2, labels
question1, question2, labels = load_data(df)
question1 = list(question1)
question2 = list(question2)
combined = question1 + question2
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,combined
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,what is the step by step guide to invest in sh...
1,1,3,4,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...,0,what is the story of kohinoor koh i noor diamo...
2,2,5,6,how can i increase the speed of my internet nn...,how can internet speed be increased by hacking...,0,how can i increase the speed of my internet nn...
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math math is divided by,0,why am i mentally very lonely how can i solve ...
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0,which one dissolve in water quikly sugar salt ...


In [ ]:
def cleanAscii(text):
       return ''.join(i for i in text if ord(i) < 128)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
max_words = 10000
tok = Tokenizer(num_words=max_words, oov_token="<OOV>")
tok.fit_on_texts(combined)
# Padding sequences to a max embedding length of 100 dim and max len of the sequence to 300
sequences = tok.texts_to_sequences(combined)
sequences = pad_sequences(sequences, maxlen=300, padding='post')

In [ ]:
import os
max_words = 10000
word_index = len(tok.word_index) + 1
glove_dir = ''
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))
print(word_index)
# matrix
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tok.word_index.items():
  if i < max_words:
    embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

Found 400000 word vectors.
79960


IndexError: ignored

In [ ]:
import tensorflow as tf
lstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))
# loading our matrix
emb = tf.keras.layers.Embedding(max_words, embedding_dim, input_length=300, weights=[embedding_matrix],trainable=False)
input1 = tf.keras.Input(shape=(300,))
e1 = emb(input1)
x1 = lstm_layer(e1)
input2 = tf.keras.Input(shape=(300,))
e2 = emb(input2)
x2 = lstm_layer(e2)
mhd = lambda x: tf.keras.backend.abs(x[0] - x[1])
merged = tf.keras.layers.Lambda(function=mhd, output_shape=lambda x: x[0],
name='L1_distance')([x1, x2])
preds = tf.keras.layers.Dense(1, activation='sigmoid')(merged)
model = tf.keras.Model(inputs=[input1, input2], outputs=preds)
model.compile(loss='mse', optimizer='adam')

NameError: ignored

Transformer Based Semantic Similarity


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.1 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=96f8341d01979985073c873cc84975df1bb19e25397feefc1174af89238051ec
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
count=0
for i in range(10000):
  sentences = [newdf['question1'][i],
    newdf['question2'][i]]
  sentence1 = sentences[0]
  sentence2 = sentences[1]
  # encode sentences to get their embeddings
  embedding1 = model.encode(sentence1, convert_to_tensor=True)
  embedding2 = model.encode(sentence2, convert_to_tensor=True)
  # compute similarity scores of two embeddings
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  if cosine_scores.item()>0.5:
    t=1
  else:
    t=0
  if t==newdf['is_duplicate'][i]:
      count+=1
  print(i)
print("accuracy=",count/10000)

Streaming output truncated to the last 5000 lines.
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190

accuracy after 10000 sentences= 60.31%

WMD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyemd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gensim
from gensim.models import Word2Vec
    
model = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/testing/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
question3 = 'Why am I mentally very lonely? How can I solve it?'
question4 = 'Find the remainder when [math]23^{24}[/math] is divided by 24,23?'
question3 = question3.lower().split()
question4 = question4.lower().split()
distance = model.wmdistance(question3, question4)
print('distance = %.4f' % distance)

ImportError: ignored

In [ ]:
model=Word2Vec(sentences=common_texts)
for i in range(10000):
  
  sentences = [newdf['question1'][i],
    newdf['question2'][i]]
  
  sentence1 = sentences[0].lower().split()
  sentence2 = sentences[1].lower().split()
  
  distance= model.wmdistance(sentence1, sentence2)

RuntimeError: ignored